### 무신사 크롤링하기

In [2]:
# 필요 라이브러리 가져오기
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import requests

In [44]:
driver = webdriver.Chrome()
driver.get('https://www.musinsa.com/categories/item/001005?d_cat_cd=001005&brand=&list_kind=small&sort=pop_category&sub_sort=&page=2&display_cnt=90&exclusive_yn='\
           + '&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=')

In [45]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [46]:
# 타이틀을 가지고 있는 클래스
pro_title = soup.select('.img-block')


##타이틀
# [i]['title']

In [47]:
pro_title_list = []
for title in pro_title:
    pro_title_list.append(title['title'])


In [48]:
len(pro_title_list)

90

In [49]:
## 브랜드
pro_brand = soup.select('.item_title > a')

## 정확한 브랜드 위치
# [i].text

In [50]:
pro_brnad_list = []
for brand in pro_brand:
    pro_brnad_list.append(brand.text)

In [51]:
len(pro_brnad_list)

90

In [52]:
## 상품 가격
pro_price = soup.select('.price')
pro_price[0].text.replace('\n','').strip().replace('원','')

'66,000                25,900'

In [53]:
pro_price_list = []
for price in pro_price:
    pro_price_list.append(price.text.replace('\n','').strip().replace('원',''))

In [54]:
pro_price_list[10].split('                ')

['69,000', '27,900']

In [60]:
df_musinsa = pd.DataFrame(data={'브랜드':pro_brnad_list,'제품명':pro_title_list})